In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

import pickle
from scipy.spatial.transform import Rotation as R

In [52]:
amass_data_folder = './amass_data_additional/'

In [53]:
paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk(amass_data_folder):
    folders.append(root)
    for name in files:
        dataset_name = root.split('/')[2]
        if dataset_name not in dataset_names and name.split(".")[-1] == "npz":
            dataset_names.append(dataset_name)
        paths.append(os.path.join(root, name))

In [54]:
save_root = './pose_data_smpl/'
save_folders = [folder.replace(amass_data_folder, './pose_data_smpl/') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]

In [55]:
len(group_path)

4

# Initialize Rotate stuff using maths

In [6]:


%cd ../motion-diffusion-model/
from model.smpl import SMPL

smpl_model = SMPL().eval()
%cd ../HumanML3D/

/coc/scratch/sanisetty3/music_motion/motion-diffusion-model
/coc/scratch/sanisetty3/music_motion/HumanML3D


In [7]:
import rotation_conversions as geometry


In [8]:
def process6D(aist_vec):
    aist_ex = torch.Tensor(aist_vec)
    trans = aist_ex[:,:3]
    if aist_vec.shape[-1] == 135:
        rots = torch.cat((aist_ex[:,3:] , aist_ex[: , -12:]) , 1) 
    else:
        rots = aist_ex[:,3:]
    aist_ex_9d = geometry.rotation_6d_to_matrix(rots.reshape(-1,6)).reshape(aist_vec.shape[0],24,3,3)
    global_orient = aist_ex_9d[:, 0:1]
    rotations = aist_ex_9d[:, 1:]
    out_aist_og = smpl_model(body_pose=rotations, global_orient=global_orient, transl = trans)

    return out_aist_og


In [9]:

def to6D(motion):
    
    if motion.shape[-1] == 225:
        motion = motion[:,6:]
    smpl_trans = motion[:,:3]
    smpl_poses = motion[:,3:]
    n_frames = smpl_poses.shape[0]
    smpl_poses = smpl_poses.reshape(n_frames , 24,3,3)
    smpl_poses_6d= geometry.matrix_to_rotation_6d(torch.Tensor(smpl_poses)).reshape(n_frames , -1)
    smpl_motion = np.concatenate([smpl_trans, smpl_poses_6d], axis=-1)
    
    return smpl_motion

In [10]:
def process9D(aist_ex):
    if aist_ex.shape[-1] == 225:
        aist_ex = aist_ex[:,6:]
    trans = aist_ex[:,:3]
    rots = aist_ex[:,3:].reshape(trans.shape[0] , -1 , 3,3)
    global_orient = rots[:, 0:1]
    rotations = rots[:, 1:]
    out_aist_og = smpl_model(body_pose=rotations, global_orient=global_orient, transl = trans)

    return out_aist_og

In [11]:
def axis_angle_to_6d(smpl_motion):
    trans = smpl_motion[:,:3]
    rots  = smpl_motion[:,3:]
    
    new_6d_rotations = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(rots.reshape(-1 , 3 ))).reshape(rots.shape[0] , -1)
    smpl_6d = torch.cat([trans , new_6d_rotations] , 1).numpy()
    
    return smpl_6d
    
            

In [12]:
def process_and_center_aist(sml , up = "y"):
    
    seq_len = sml.shape[0]

    transl = sml[..., :, :3]
    rot_6D = sml[..., :, 3:].reshape( seq_len, -1, 6)

    if sml.shape[-1] == 135:
        rot_6D = np.concatenate((rot_6D , rot_6D[:,-2:]) , -2)

    rotmats = geometry.rotation_6d_to_matrix(torch.Tensor(rot_6D))
    
    smpl_output = smpl_model.forward(
        global_orient=torch.Tensor(rotmats[: , :1]),
        body_pose=rotmats[: , 1:],
        transl=torch.Tensor(transl),)
    
    positions_kyp = smpl_output["smpl"].numpy()
    
    '''Put on Floor'''
    floor_height = positions_kyp.min(axis=0).min(axis=0)[1]
    
    positions_ = sml[:, :3]

    positions_[:,1]  -= floor_height
    '''XZ at origin'''
    root_pos_init = positions_[0]
    root_pose_init_xy = root_pos_init * np.array([1, 0, 1])
    positions_ = positions_ - root_pose_init_xy
    
    sml[:, :3]  = positions_
    
    return sml

In [13]:
import torch
from tqdm import tqdm
import plotly.express as px
import pandas as pd
def show_cloud(vertices):
    indexed_vertices_mod = np.concatenate((np.arange(vertices.shape[0]).reshape(-1,1),vertices),axis=1)

    vertices_mod_df = pd.DataFrame(indexed_vertices_mod)
    vertices_mod_df.columns = ['idx','x','y','z']

    fig = px.scatter_3d(vertices_mod_df,x='x',y='y',z='z',hover_data=['idx'],)
    fig.update_traces(marker=dict(size=4,line=dict(width=4,color='DarkSlateGrey')))
    fig.show()

In [18]:
def load_smpl_axis_angle(file_path , target_fps):
    raw_amass_motion = np.load(file_path, allow_pickle = True)
    fps = 0
    try:
        if "GRAB" in file_path or "SOMA" in file_path:
            fps = raw_amass_motion['mocap_frame_rate']
        else:
            fps = raw_amass_motion['mocap_framerate']
        frame_number = raw_amass_motion['trans'].shape[0]
    except:
        return None
    
    down_sample = max(int(fps / target_fps) , 1)
    frame_number = raw_amass_motion['trans'].shape[0]
    pose_seq = []
    with torch.no_grad():
        for fId in range(0, frame_number, down_sample):
            axis_angle_rots = torch.Tensor(raw_amass_motion['poses'][fId:fId+1, :66])
            trans = torch.Tensor(raw_amass_motion['trans'][fId:fId+1])
            motion_tensor = torch.cat((trans , axis_angle_rots) , 1)
            pose_seq.append(motion_tensor)
    pose_seq = torch.cat(pose_seq, dim=0).detach().cpu()
    
    return pose_seq, fps

In [15]:
def rotate_trans(translation):
    r = R.from_euler('XYZ', [0,180,0], degrees=True)
    return translation[:,[2,1,0]][:,[1,0,2]]@np.round(r.as_matrix()).T


In [16]:
def rotate_smpl_to(input_smpl, target_quat):
    trans = input_smpl[:,:3]
    rots = input_smpl[:,3:] ##axis angle
    
    joint_quat_rotations = geometry.axis_angle_to_quaternion(rots.reshape(-1,3)).reshape(trans.shape[0] , 22 , 4)
    root_quat_rotations = joint_quat_rotations[:,0]
    offset_quat_rotation = geometry.quaternion_multiply(torch.Tensor(target_quat) , geometry.quaternion_invert(root_quat_rotations[0]))
    rotated_all_root_quat = geometry.quaternion_multiply(offset_quat_rotation , root_quat_rotations)
    
    
    rots[:,:3] = geometry.quaternion_to_axis_angle(rotated_all_root_quat)
    new_6d_rotations = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(rots.reshape(-1 , 3 ))).reshape(rots.shape[0] , -1)
    
    trans = rotate_trans(trans )
    final_rotated_smpl = torch.cat([trans , new_6d_rotations] , 1).numpy()

    
    return final_rotated_smpl

In [120]:
def rotate_smpl_by(input_smpl_6d, y_rot):
    input_smpl = torch.Tensor(input_smpl_6d)
    r = R.from_euler('XYZ', [0,y_rot,0], degrees=True)
    r_matrix = torch.Tensor(r.as_matrix())
    r_quat = torch.Tensor(r.as_quat()[[3,0,1,2]])
    
    
    trans = input_smpl[:,:3]
    
    root_orient = input_smpl[:,3:9]
    root_orient_quat = geometry.matrix_to_quaternion(geometry.rotation_6d_to_matrix(root_orient)).reshape(-1,4)
    rotated_all_root_quat = geometry.quaternion_multiply(r_quat , root_orient_quat)
    
    
    input_smpl[:,3:9] = geometry.matrix_to_rotation_6d(geometry.quaternion_to_matrix(rotated_all_root_quat))
    
    input_smpl[:,:3] = trans@r_matrix.T

    
    return input_smpl

## Process AMASS to SMPL

In [56]:

ex_fps = 20
def amass_to_smpl(src_path, save_path):
    
    amass_smpl, fps = load_smpl_axis_angle(src_path, ex_fps)
    amass_rotated_smpl_6d = rotate_smpl_to(amass_smpl.clone() , [1,0,0,0])
    amass_centered_smpl_6d = process_and_center_aist(amass_rotated_smpl_6d)

    np.save(save_path, amass_centered_smpl_6d)
    return fps

In [57]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

In [58]:
all_count

1752

In [67]:
import time
for paths in [group_path[0] , group_path[1] , group_path[3]]:
    dataset_name = paths[0].split('/')[2]
    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
#         name =  path[:-3]
#         save_path = f"/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D_SMPL/new_joints_vec/{path[:-3]}.npy"
        save_path = path.replace(amass_data_folder, './pose_data_smpl/')
        save_path = save_path[:-3] + 'npy'
        
        try:
            fps = amass_to_smpl(path, save_path)
        except Exception as e:
            print(e)
            print(path)
            continue
        
    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)

Processing: HUMAN4D:   1%|▏         | 2/156 [00:25<33:48, 13.17s/it]

cannot unpack non-iterable NoneType object
./amass_data_additional/HUMAN4D/Subject3_4_Medhi/shape.npz


Processing: HUMAN4D:  12%|█▏        | 18/156 [02:47<25:54, 11.26s/it]

cannot unpack non-iterable NoneType object
./amass_data_additional/HUMAN4D/Subject1_2_Karim/shape.npz


Processing: HUMAN4D:  23%|██▎       | 36/156 [04:18<04:48,  2.40s/it]

...

cannot unpack non-iterable NoneType object
./amass_data_additional/SOMA/soma_subject1/male_stagei.npz


Processing: SOMA:  67%|██████▋   | 48/72 [01:30<00:31,  1.33s/it]

cannot unpack non-iterable NoneType object
./amass_data_additional/SOMA/soma_subject2/male_stagei.npz


Processing: SOMA: 100%|██████████| 72/72 [02:03<00:00,  1.72s/it]


Processed / All (fps 120): 1752/1752


In [68]:
len(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/pose_data_smpl/*/*/*.npy"))

15653

In [50]:
len(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/pose_data/*/*/*.npy"))

16844

In [47]:
len(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/pose_data_smpl/*/*/*.npy"))

13943

In [ ]:
BMLmovi
BMLhandball

## Segment, Mirror and Relocate Motions

In [70]:
def swap_left_right_smpl(data_):
    data = data_.copy()
    trans = data[: ,:3]
    rots = data[: , 3:].reshape(-1  , 22 , 6)
    trans[..., 0] *= -1
    rots[:,0] = (-1*rots[:,0])
    return np.concatenate((trans , rots.reshape(trans.shape[0] , -1)) , 1)

In [ ]:
np.load

In [119]:
mirror = rotate_smpl_by(np.load(index_file.loc[0]['source_path'].replace("pose_data" , "pose_data_smpl")) , 180)

AttributeError: 'numpy.ndarray' object has no attribute 'norm'

In [80]:
mirror.shape

(117, 135)

In [81]:
np.save("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/test/000000_m.npy" , mirror)

In [71]:
index_path = './index.csv'
save_dir = './joints_smpl'
index_file = pd.read_csv(index_path)
total_amount = index_file.shape[0]
fps = 20

(117, 135)

In [ ]:
for i in tqdm(range(total_amount)):
    source_path = index_file.loc[i]['source_path'].replace("pose_data" , "pose_data_smpl")
    new_name = index_file.loc[i]['new_name']
    try:
        data = np.load(source_path)
    except:
        print(source_path.split("/")[2])
        continue
    start_frame = index_file.loc[i]['start_frame']
    end_frame = index_file.loc[i]['end_frame']
    if 'humanact12' not in source_path:
        if 'Eyes_Japan_Dataset' in source_path:
            data = data[3*fps:]
        if 'MPI_HDM05' in source_path:
            data = data[3*fps:]
        if 'TotalCapture' in source_path:
            data = data[1*fps:]
        if 'MPI_Limits' in source_path:
            data = data[1*fps:]
        if 'Transitions_mocap' in source_path:
            data = data[int(0.5*fps):]
        data = data[start_frame:end_frame]
        data[..., 0] *= -1
    
    data_m = swap_left_right2(data)
#     save_path = pjoin(save_dir, )
    np.save(pjoin(save_dir, new_name), data)
    np.save(pjoin(save_dir, 'M'+new_name), data_m)

In [ ]:
total_amount = len(glob("joints_smpl/*.npy"))

In [ ]:
save_dir = './joints_smpl/'

new_name = total_amount-1
cnt = 0
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    for path in pbar:
        load_path = path.replace(amass_data_folder, './pose_data_smpl/').replace("npz" , "npy")
        try:
            data = np.load(load_path)
            if data.shape[-1] != 135:
                print(load_path , data.shape)
        except:
            print(load_path)
            continue
        new_name +=1 
        cnt+=1
        
        if 'Dance' in load_path:
            if data.shape[0] > 1000:
                data = data[6*fps:]
                            
        data_m = swap_left_right_smpl(data)
        np.save(pjoin(save_dir, str(new_name).zfill(6) + ".npy"), data)
        np.save(pjoin(save_dir, 'M'+str(new_name).zfill(6) + ".npy"), data_m)
   


## Render hml_vec

In [30]:
from glob import glob
import numpy as np
import torch
import time 


In [31]:
%cd ../motion_vqvae
from render_final import hml2aist
from utils.motion_process import recover_from_ric
import visualize.plot_3d_global as plot_3d

%cd ../HumanML3D/

/coc/scratch/sanisetty3/music_motion/motion_vqvae
/coc/scratch/sanisetty3/music_motion/HumanML3D


In [10]:

def to_xyz(motion, mean ,std , j = 22):
    motion_xyz = recover_from_ric(motion.cpu().float()*std+mean, j)
    motion_xyz = motion_xyz.reshape(motion.shape[0],-1, j, 3)
    return motion_xyz

            
def sample_render(motion_xyz , name , save_path):
    print(f"render start")
    
    gt_pose_vis = plot_3d.draw_to_batch(motion_xyz.numpy(),None, [os.path.join(save_path,name + ".gif")])

mean = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D/Mean.npy")
std = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D/Std.npy")

In [5]:
new_joint_vec_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D/new_joint_vecs/000000.npy"
new_joint_vec = np.load(new_joint_vec_path)

In [27]:
a = time.time()
xyz = to_xyz(torch.Tensor(new_joint_vec[None,...]) , mean, std)
aist_rep = hml2aist(xyz[0].numpy())
b = time.time()

cuda:0
./body_models/
Running SMPLify, it may take a few minutes.
torch.Size([116, 3, 3]) torch.Size([116, 23, 3, 3]) torch.Size([1, 3, 116])


In [46]:
def process9D(aist_ex):
    aist_ex = torch.Tensor(aist_ex)
    if aist_ex.shape[-1] == 225:
        aist_ex = aist_ex[:,6:]
    trans = aist_ex[:,:3]
    rots = aist_ex[:,3:].reshape(trans.shape[0] , -1 , 3,3)
    global_orient = rots[:, 0:1]
    rotations = rots[:, 1:]
    out_aist_og = smpl_model(body_pose=rotations, global_orient=global_orient, transl = trans)

    return out_aist_og

In [49]:
smpl_hml  = process9D(aist_rep)

In [60]:
verts =smpl_hml["vertices"]
show_cloud(verts[80])

In [57]:
np.save("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/joints_new_rot/hml_000000.npy" , to6D(aist_rep)[: , :-12])